In [1]:
# Makes print and division act like Python 3
from __future__ import print_function, division

# Import the usual libraries
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime

import pandas as pd

# seaborn package for making pretty plots, but not necessary
try:
    import seaborn as sns
    params =   {'xtick.direction': 'in', 'ytick.direction': 'in', 'font.family': ['serif'],
                'text.usetex': True, 'text.latex.preamble': ['\usepackage{gensymb}']}
    sns.set_style("ticks", params)
except ImportError:
    print('Seaborn module is not installed.')

from IPython.display import display, Latex

from astropy.io import fits
import nghxrg as ng

In [2]:
# Enable inline plotting
%matplotlib inline
# I prefer the image display origin in the lower left
matplotlib.rcParams['image.origin'] = 'lower'

In [3]:
# Set log output levels
# webbpsf and poppy have too many unnecessary warnings
import logging
logging.getLogger('nghxrg').setLevel(logging.ERROR)

_log = logging.getLogger('ngNRC')
#_log.setLevel(logging.DEBUG)
_log.setLevel(logging.INFO)
#_log.setLevel(logging.WARNING)
#_log.setLevel(logging.ERROR)
logging.basicConfig(level=logging.WARNING,format='%(name)-10s: %(levelname)-8s %(message)s')

In [4]:
def SCAnoise(scaid, inst_params=None, file_out=None):
    """
    Create a data cube consisting of realistic NIRCam detector noise.
    """

    if inst_params is None:
        _log.warn('No parameter set passed to validate_readout(). Returning...')
        return

    # Extract all values to variables w/ key names to use throughout
    for key,val in inst_params.items():
        exec(key + '=val')
    wind_mode = wind_mode.upper(); read_mode = read_mode.upper()

    # naxis and channel info for different readout modes
    mode_arr = ['FULL', 'STRIPE', 'WINDOW']
    xpix_arr = np.array([2048., 2048, xpix])
    ypix_arr = np.array([2048., ypix, ypix])
    nch_arr = np.array([4.,4,1])
    chs_arr = xpix_arr / nch_arr

    # Determine how many lines get added if subarray (WINDOW) mode
    sub = 2**(np.arange(8)+4.)
    l1 = np.array([3,2.5,2,2,2,2,2,2])
    lsub_add = ((np.interp([xpix],sub,l1)+0.5).astype(int))[0]

    # Number of lines/rows added to a given frame, depending on mode
    line_arr = np.array([1,1,lsub_add])
    pix_offset = np.array([1,1,0]) # Add a single pix offset for full frame and stripe.

    # Number of overhead pixels per line
    nroh = 12.

    # Select info for specified readout mode
    if wind_mode in mode_arr:
        ind = mode_arr.index(wind_mode)
        nfoh = line_arr[ind]
        nfoh_pix = pix_offset[ind]
        naxis1 = xpix_arr[ind]
        naxis2 = ypix_arr[ind]
        n_out = nch_arr[ind]
    else:
        _log.warn('Window mode %s not recognized in wind_mode.' % wind_mode)
        _log.warn('Only FULL, STRIPE, and WINDOW are allowed.')

    # How many total frames (incl. dropped and all) per ramp?
    # Exclude last set of nd2 and nd3 (drops that add nothing)
    naxis3 = nd1 + ngroup*nf + (ngroup-1)*nd2

    sca_str = np.str(scaid)
    base_dir  = '/Volumes/NIRData/NIRCam/Char_Darks/CV3/'
    bias_file = base_dir + sca_str+'/SUPER_BIAS_'+sca_str+'.FITS'
    dark_file = base_dir + sca_str+'/SUPER_DARK_'+sca_str+'.FITS'
    
    # Instantiate a noise generator object
    ng_h2rg = ng.HXRGNoise(naxis1=naxis1, naxis2=naxis2, naxis3=naxis3, n_out=n_out,
                 nroh=nroh, nfoh=nfoh, nfoh_pix=nfoh_pix,
                 dark_file=dark_file, bias_file=bias_file, verbose=False,
                 wind_mode=wind_mode, x0=x0, y0=y0)


    # Lists of each SCA and their corresponding noise info
    # These come from measured dark ramps acquired during CV3 at GSFC
    sca_arr = (np.arange(10)+481).tolist()
    # Gain values (e/ADU). Everything else will be in measured ADU units
    gn_arr =  [2.07, 2.01, 2.16, 2.01, 1.83, 
               2.00, 2.42, 1.93, 2.30, 1.85]
    # Noise Values
    ktc_arr = [18.5, 15.9, 15.2, 16.9, 20.0, 
               19.2, 16.1, 19.1, 19.0, 20.0]
    ron_arr = [ 5.0,  4.3,  4.0,  4.4,  4.1,  
                5.1,  4.4,  5.0,  4.3,  4.5]
    cp_arr  = [ 4.3,  2.8,  3.0,  3.0,  4.0,   
                4.3,  4.6,  4.8,  4.8,  5.0]
    up_arr  = [[0.9,0.9,0.9,1.0], [0.9,1.0,0.9,1.0], [1.0,0.9,1.0,1.0], [0.8,0.9,0.9,0.8],
               [1.0,1.3,1.0,1.1],
               [1.0,0.9,1.0,1.0], [0.9,0.9,1.1,1.0], [1.0,1.0,1.0,0.9], [1.1,1.1,0.8,0.9],
               [1.1,1.1,1.0,1.0]]
    # Offset Values
    bias_avg_arr = [5900, 5400, 6400, 6150, 11650, 
                    7300, 7500, 6700, 7500, 11500]
    bias_sig_arr = [20.0, 20.0, 30.0, 11.0, 50.0, 
                    20.0, 20.0, 20.0, 20.0, 20.0]
    ch_off_arr   = [[1700, 530, -375, -2370], [-150, 570, -500, 350], [-530, 315, 460, -200],
                    [480, 775, 1040, -2280],  [560, 100, -440, -330],
                    [105, -29, 550, -735],    [315, 425, -110, -590],   [918, -270, 400, -1240],
                    [-100, 500, 300, -950],   [-35, -160, 125, -175]]
    f2f_corr_arr = [14.0, 13.8, 27.0, 14.0, 26.0,
                    14.7, 11.5, 18.4, 14.9, 14.8]
    f2f_ucorr_arr= [[18.4,11.1,10.8,9.5], [7.0,7.3,7.3,7.1], [6.9,7.3,7.3,7.5],
                    [6.9,7.3,6.5,6.7], [16.6,14.8,13.5,14.2],
                    [7.2,7.5,6.9,7.0], [7.2,7.6,7.5,7.4], [7.9,6.8,6.9,7.0],
                    [7.6,8.6,7.5,7.4], [13.3,14.3,14.1,15.1]]
    aco_a_arr    = [[770, 440, 890, 140], [800, 410, 840, 800], [210,680,730,885],
                    [595, 642, 634, 745], [-95,660,575,410],
                    [220, 600, 680, 665], [930,1112, 613, 150], [395, 340, 820, 304],
                    [112, 958, 690, 907], [495, 313, 392, 855]]
    ref_inst_arr = [1.0, 1.5, 1.0, 1.3, 1.0, 
                    1.0, 1.0, 1.0, 2.2, 1.0]


    # SCA Index
    ind = sca_arr.index(scaid)

    gn = gn_arr[ind]
    # Noise Values
    ktc_noise= gn * ktc_arr[ind] * 1.1   # kTC noise in electrons
    rd_noise = gn * ron_arr[ind] * 0.95  # White read noise per integration
    c_pink   = gn * cp_arr[ind] * 1.3    # Correlated pink noise
    # Uncorrelated pink noise
    u_pink   = gn * np.array(up_arr[ind]) * 1.3
    ref_rat  = 0.9        # Ratio of reference pixel noise to that of reg pixels

    # Offset Values
    bias_off_avg = gn * bias_avg_arr[ind] + 110  # On average, integrations start here in electrons
    bias_off_sig = gn * bias_sig_arr[ind]  # bias_off_avg has some variation. This is its std dev.
    bias_amp     = gn * 1.0     # A multiplicative factor to multiply bias_image. 1.0 for NIRCam.

    # Offset of each channel relative to bias_off_avg.
    ch_off = gn * np.array(ch_off_arr[ind]) + 110
    # Random frame-to-frame reference offsets due to PA reset
    ref_f2f_corr  = gn * f2f_corr_arr[ind] * 0.95
    ref_f2f_ucorr = gn * np.array(f2f_ucorr_arr[ind]) * 1.15 # per-amp
    # Relative offsets of altnernating columns
    aco_a = gn * np.array(aco_a_arr[ind])
    aco_b = -1 * aco_a
    #Reference Instability
    ref_inst = gn * ref_inst_arr[ind]

    # If only one output (window mode) then select first elements of each array
    if n_out == 1:
        u_pink = u_pink[0]
        ch_off = ch_off[0]
        ref_f2f_ucorr = ref_f2f_ucorr[0]
        aco_a = aco_a[0]; aco_b = aco_b[0]

    # Run noise generator
    out_ADU = False if file_out is None else True
    hdu = ng_h2rg.mknoise(file_out, gain=gn, rd_noise=rd_noise, c_pink=c_pink, u_pink=u_pink, 
            reference_pixel_noise_ratio=ref_rat, ktc_noise=ktc_noise,
            bias_off_avg=bias_off_avg, bias_off_sig=bias_off_sig, bias_amp=bias_amp,
            ch_off=ch_off, ref_f2f_corr=ref_f2f_corr, ref_f2f_ucorr=ref_f2f_ucorr, 
            aco_a=aco_a, aco_b=aco_b, ref_inst=ref_inst, out_ADU=out_ADU)

    #return hdu


In [5]:
inst_params = {'filter': 'F444W', 'mask': None, 'pupil': None, 'module': 'A',
               'xpix': 2048, 'ypix': 2048, 'x0':0, 'y0':0, 
               'read_mode':'RAPID', 'wind_mode': 'FULL', 'cnaw_filt': False,
               'nramps': 1, 'ngroup': 108, 'nf': 1, 'nd1': 0, 'nd2': 0, 'nd3': 0}


niter = 15
base_dir  = '/Volumes/NIRData/NIRCam/Char_Darks/CV3/'
base_dir = '/data/jarronl/sim_CV3_darks/'
outdir = base_dir + 'FITS/480/'
name_480 = ['NRCNRCA0-DARK-',60000000000,'_1_480_SE_']
#NRCNRCA1-DARK-60090604481_1_481_SE_2016-01-09T06h52m47.fits
for i in range(niter):
    now = datetime.datetime.now().isoformat()[:-7]
    now = now.replace(':', 'h', 1)
    now = now.replace(':', 'm', 1)
    print(i,now);
    name_480[1] += 1
    file_out = outdir + name_480[0] + str(name_480[1]) + name_480[2] + now + '.fits'
    SCAnoise(486, inst_params, file_out)

0 2016-05-06T11h11m41
1 2016-05-06T11h18m50
2 2016-05-06T11h25m52
3 2016-05-06T11h32m54
4 2016-05-06T11h39m59
5 2016-05-06T11h47m08
6 2016-05-06T11h54m16
7 2016-05-06T12h01m24
8 2016-05-06T12h08m32
9 2016-05-06T12h15m39
10 2016-05-06T12h22m48
11 2016-05-06T12h29m55
12 2016-05-06T12h37m02
13 2016-05-06T12h44m10
14 2016-05-06T12h51m17


In [5]:
import datetime
from ngNRC import *

inst_params = {'filter': 'F444W', 'mask': None, 'pupil': None, 'module': 'A',
               'xpix': 2048, 'ypix': 2048, 'x0':0, 'y0':0, 
               'read_mode':'RAPID', 'wind_mode': 'FULL', 'cnaw_filt': False,
               'nramps': 1, 'ngroup': 108, 'nf': 1, 'nd1': 0, 'nd2': 0, 'nd3': 0}

sca_arr = (np.arange(10)+481).tolist()
for sca in sca_arr:
    sca_str = np.str(sca)
    niter = 10
    base_dir = '/data/jarronl/sim_CV3_darks/'
    outdir = base_dir + 'FITS/'+sca_str+'/'
    fname = ['NRCNRCA0-DARK-',60000000000,'_1_'+sca_str+'_SE_']
    for i in range(niter):
        now = datetime.datetime.now().isoformat()[:-7]
        now = now.replace(':', 'h', 1)
        now = now.replace(':', 'm', 1)
        print(i,now);
        fname[1] += 1
        file_out = outdir + fname[0] + str(fname[1]) + fname[2] + now + '.fits'
        hdu = SCAnoise(sca, inst_params, file_out)
        del hdu
        